# 中文文本分词
以THUCnews为数据集（该数据集已经过筛选，并非完整数据集，文件中以**filtered_cnews.train.txt**来标识。

## 1.朴素贝叶斯方法
首先导入相应的库

In [1]:
import pandas as pd
import numpy as np
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import f1_score

读取并预处理数据

In [2]:
data = pd.read_csv('filtered_cnews.train.txt', sep='\t', header=None, names=['category', 'content'], encoding='utf-8')
print("数据预览：")
print(data.head())

数据预览：
  category                                            content
0       体育  马晓旭意外受伤让国奥警惕 无奈大雨格外青睐殷家军记者傅亚雨沈阳报道 来到沈阳，国奥队依然没有...
1       体育  商瑞华首战复仇心切 中国玫瑰要用美国方式攻克瑞典多曼来了，瑞典来了，商瑞华首战求3分的信心也...
2       体育  冠军球队迎新欢乐派对 黄旭获大奖张军赢下PK赛新浪体育讯12月27日晚，“冠军高尔夫球队迎新...
3       体育  辽足签约危机引注册难关 高层威逼利诱合同笑里藏刀新浪体育讯2月24日，辽足爆发了集体拒签风波...
4       体育  揭秘谢亚龙被带走：总局电话骗局 复制南杨轨迹体坛周报特约记者张锐北京报道  谢亚龙已经被公安...


划分数据集为训练集、验证集、测试集，random_state是随机的种子，是为了保证随机的结果可以复现，方便debug

In [3]:
train_val, test = train_test_split(data, test_size=0.2, random_state=42) # 80%训练集和20%测试集
train, val = train_test_split(train_val, test_size=0.25, random_state=42)  # 80% * 25% = 20%验证集



特征提取（数据量较小，所用使用TF-IDF方法）

In [4]:
vectorizer = TfidfVectorizer(
    tokenizer=jieba.cut,  # 使用jieba对文本进行分词
    max_features=10000  # 特征数量为10000
)
#训练集以内容为自变量，分类为因变量
X_train = vectorizer.fit_transform(train['content'])  
y_train = train['category']  
#验证集以内容为自变量，分类为因变量
X_val = vectorizer.transform(val['content'])  
y_val = val['category']  
#测试集以内容为自变量，分类为因变量
X_test = vectorizer.transform(test['content'])  
y_test = test['category']  

c:\Users\Skye\anaconda3\Lib\site-packages\sklearn\feature_extraction\text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Skye\AppData\Local\Temp\jieba.cache
Loading model cost 0.485 seconds.
Prefix dict has been built successfully.


训练朴素贝叶斯模型

In [5]:
model = MultinomialNB()  # 初始化多项式朴素贝叶斯分类器
model.fit(X_train, y_train)  # 使用训练集特征和标签训练模型

MultinomialNB()

生成验证集评估结果

In [6]:
y_pred_val = model.predict(X_val)  # 使用模型对验证集进行预测
print("\n验证集结果：")
print(classification_report(y_val, y_pred_val))  # 打印验证集的分类报告，包括精确率、召回率和F1分数



验证集结果：
              precision    recall  f1-score   support

          体育       1.00      0.99      1.00      1014
          家居       0.97      0.93      0.95      1007
          房产       0.91      0.95      0.93       951
          教育       0.92      0.95      0.93      1026
          科技       0.93      0.96      0.94       953
          财经       0.96      0.92      0.94      1049

    accuracy                           0.95      6000
   macro avg       0.95      0.95      0.95      6000
weighted avg       0.95      0.95      0.95      6000



生成测试集评估结果：

In [7]:
y_pred_test = model.predict(X_test)  # 使用模型对测试集进行预测
print("\n测试集结果：")
print(classification_report(y_test, y_pred_test))  # 打印测试集的分类报告



测试集结果：
              precision    recall  f1-score   support

          体育       0.99      0.99      0.99       978
          家居       0.96      0.93      0.94      1024
          房产       0.92      0.95      0.93      1046
          教育       0.91      0.93      0.92       950
          科技       0.94      0.96      0.95      1033
          财经       0.95      0.91      0.93       969

    accuracy                           0.94      6000
   macro avg       0.95      0.94      0.94      6000
weighted avg       0.95      0.94      0.94      6000



提取宏平均和微平均指标

In [8]:
macro_f1 = f1_score(
    y_test,  # 测试集真实标签
    y_pred_test,  # 测试集预测标签
    average='macro'  # 计算宏平均F1分数
)
micro_f1 = f1_score(
    y_test,  
    y_pred_test,  
    average='micro'  # 计算微平均F1分数
)
accuracy = accuracy_score(
    y_test,  # 测试集真实标签
    y_pred_test  # 测试集预测标签
)

得出最终结果：

In [9]:
print(f"\n最终结果：")
print(f"准确率：{accuracy:.4f}")  # 打印测试集的准确率
print(f"宏平均F1: {macro_f1:.4f}")  # 打印宏平均F1分数
print(f"微平均F1: {micro_f1:.4f}")  # 打印微平均F1分数


最终结果：
准确率：0.9448
宏平均F1: 0.9449
微平均F1: 0.9448


## 2.KNN方法

导入库

In [10]:
import pandas as pd
import numpy as np
import jieba
import logging
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler  

禁用jieba的日志输出，避免分词时打印多余信息

In [11]:
logging.getLogger('jieba').setLevel(logging.ERROR)

读取并预处理数据、数据划分、特征提取(上面已经完成，此处略)

标准化特征，虽然TF-IDF已经是标准化的，但KNN对距离敏感，进一步标准化可能提升效果

In [12]:
scaler = StandardScaler()  # 初始化标准化器
# 对训练集特征进行标准化
X_train = scaler.fit_transform(X_train.toarray())
# 对验证集特征进行标准化（使用训练集的标准化参数）
X_val = scaler.transform(X_val.toarray())
# 对测试集特征进行标准化（使用训练集的标准化参数）
X_test = scaler.transform(X_test.toarray())

训练KNN模型

In [13]:
# 初始化KNN分类器，设置邻居数n_neighbors=5，使用余弦相似度作为距离度量
model = KNeighborsClassifier(n_neighbors=5, metric='cosine')
# 使用训练集数据训练KNN模型
model.fit(X_train, y_train)

KNeighborsClassifier(metric='cosine')

验证集评估

In [14]:
y_pred_val = model.predict(X_val)
# 打印验证集的分类报告，包括精确率、召回率和F1分数
print("\n验证集结果：")
print(classification_report(y_val, y_pred_val))


验证集结果：
              precision    recall  f1-score   support

          体育       0.95      0.98      0.97      1014
          家居       0.88      0.90      0.89      1007
          房产       0.85      0.89      0.87       951
          教育       0.92      0.86      0.89      1026
          科技       0.88      0.87      0.88       953
          财经       0.91      0.90      0.91      1049

    accuracy                           0.90      6000
   macro avg       0.90      0.90      0.90      6000
weighted avg       0.90      0.90      0.90      6000



测试集评估

In [15]:
y_pred_test = model.predict(X_test)
# 打印测试集的分类报告，包括精确率、召回率和F1分数
print("\n测试集结果：")
print(classification_report(y_test, y_pred_test))


测试集结果：
              precision    recall  f1-score   support

          体育       0.95      0.98      0.96       978
          家居       0.85      0.90      0.87      1024
          房产       0.87      0.90      0.88      1046
          教育       0.89      0.84      0.86       950
          科技       0.90      0.89      0.89      1033
          财经       0.92      0.88      0.90       969

    accuracy                           0.90      6000
   macro avg       0.90      0.90      0.90      6000
weighted avg       0.90      0.90      0.90      6000



提取宏平均和微平均指标

In [16]:
# 计算测试集的宏平均F1分数
macro_f1 = f1_score(y_test, y_pred_test, average='macro')
# 计算测试集的微平均F1分数
micro_f1 = f1_score(y_test, y_pred_test, average='micro')
# 计算测试集的准确率
accuracy = accuracy_score(y_test, y_pred_test)

最终结果：

In [17]:
print(f"\n最终结果：")
print(f"准确率：{accuracy:.4f}")
print(f"宏平均F1: {macro_f1:.4f}")
print(f"微平均F1: {micro_f1:.4f}")


最终结果：
准确率：0.8967
宏平均F1: 0.8967
微平均F1: 0.8967
